In [2]:
import os, shutil
import tensorflow.keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from tensorflow.keras import initializers
from tensorflow.keras import callbacks
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import array_to_img

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pydicom
from PIL import Image
from PIL.Image import core as _imaging
import cv2
import shutil
import sys
import gdcm
from skimage import exposure


In [3]:
root_directory = "C:/Users/Mojtaba Jafari Tadi/Projects_AMK/Radiology Project"

code_dirs = os.chdir( root_directory+"/fatpad" )
os.getcwd()

'C:\\Users\\Mojtaba Jafari Tadi\\Projects_AMK\\Radiology Project\\fatpad'

In [4]:
labels = pd.read_excel('./Labels_FatPadDetection.xlsx', engine='openpyxl')
new=labels.set_index('Patient ID')


In [5]:
base_dir1_clahe = '/reformatted_data_clahe/No_Effusion_No_Fracture'
base_dir2_clahe = '/reformatted_data_clahe/Yes_Effusion_No_Fracture'
base_dir3_clahe = '/reformatted_data_clahe/No_Effusion_Yes_Fracture'
base_dir4_clahe = '/reformatted_data_clahe/Yes_Effusion_Yes_Fracture'

if not os.path.exists(root_directory+base_dir1_clahe):
    os.makedirs(root_directory+base_dir1_clahe)
if not os.path.exists(root_directory+base_dir2_clahe):
    os.makedirs(root_directory+base_dir2_clahe)
if not os.path.exists(root_directory+base_dir3_clahe):
    os.makedirs(root_directory+base_dir3_clahe)
if not os.path.exists(root_directory+base_dir4_clahe):
    os.makedirs(root_directory+base_dir4_clahe)

In [6]:
### switch to main directory
dicom_dir="/data_DICOM/"
dirs = os.chdir(root_directory)
os.getcwd()

'C:\\Users\\Mojtaba Jafari Tadi\\Projects_AMK\\Radiology Project'

In [7]:
#unpack the directory of images which is named radio_images_dataset into a similar named directory
#the walk stops the moment it encounters anything that is not a dicom image
IMG_SIZE=600
#p2, p98 = np.percentile(img, (2, 98))
#img_rescale = exposure.rescale_intensity(img, in_range=(p2, p98))

for dirpath, dirnames, files in os.walk('./data_DICOM', topdown=False):
    print(dirpath)
    dir_name = dirpath.split(os.path.sep)[-1]
    if new.loc[int(dir_name)][0]==0 and new.loc[int(dir_name)][1]==0:
        for file_name in files:
            #print('file_name no effusion no fatpad')
            ds = pydicom.dcmread(dirpath+'/'+file_name)
            img = ds.pixel_array
            img2d = img.astype(float)
            img2dscaled = (np.maximum(img2d,0)/img2d.max())*255.0
            img2dscaled = np.uint8(img2dscaled)
            

            #using ds,pixel_array because the pydicom documentation says it will automatically take care of compressed jpg or png,
            #pixel data and also encapsulated pixel data multi-frame volumes in the dicom image
            #cv2 was not mentioned in the pydicom documentation but it seems to work well as seen above
            #png supports 16 gray scale and does not require the PILOW workaround for jpg images
            
            new_img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            image_bw_arr = np.asarray(new_img)
            clahe = cv2.createCLAHE(clipLimit = 5) 
            clh_img = clahe.apply(image_bw_arr) + 10
            #equ = exposure.equalize_hist(clh_img)
            #final_img=255-equ
            final_img = np.reshape(clh_img, (IMG_SIZE, IMG_SIZE,1))
            final_img = array_to_img(final_img)
            if not os.path.exists(root_directory+base_dir1_clahe+'/'+file_name+'_clahe.jpg'):
                final_img.save(root_directory+base_dir1_clahe+'/'+file_name+'_clahe.jpg')
                
    if new.loc[int(dir_name)][0]==1 and new.loc[int(dir_name)][1]==0:
        for file_name in files:
            #print('file_name yes effusion no fatpad')
            ds = pydicom.dcmread(dirpath+'/'+file_name)
            img = ds.pixel_array
            img2d = img.astype(float)
            img2dscaled = (np.maximum(img2d,0)/img2d.max())*255.0
            img2dscaled = np.uint8(img2dscaled)
        

            #using ds,pixel_array because the pydicom documentation says it will automatically take care of compressed jpg or png,
            #pixel data and also encapsulated pixel data multi-frame volumes in the dicom image
            #cv2 was not mentioned in the pydicom documentation but it seems to work well as seen above
            #png supports 16 gray scale and does not require the PILOW workaround for jpg images
            new_img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            image_bw_arr = np.asarray(new_img)
            clahe = cv2.createCLAHE(clipLimit = 5) 
            clh_img = clahe.apply(image_bw_arr) + 10
            #equ = exposure.equalize_hist(clh_img)
            #final_img=255-equ
            final_img = np.reshape(clh_img, (IMG_SIZE, IMG_SIZE,1))
            final_img = array_to_img(final_img)
            if not os.path.exists(root_directory+base_dir2_clahe+'/'+file_name+'_clahe.jpg'):
                final_img.save(root_directory+base_dir2_clahe+'/'+file_name+'_clahe.jpg')
    if new.loc[int(dir_name)][0]==0 and new.loc[int(dir_name)][1]==1:
        for file_name in files:
            #print('file_name no effusion yes fatpad')
            ds = pydicom.dcmread(dirpath+'/'+file_name)
            img = ds.pixel_array
            img2d = img.astype(float)
            img2dscaled = (np.maximum(img2d,0)/img2d.max())*255.0
            img2dscaled = np.uint8(img2dscaled)
        

            #using ds,pixel_array because the pydicom documentation says it will automatically take care of compressed jpg or png,
            #pixel data and also encapsulated pixel data multi-frame volumes in the dicom image
            #cv2 was not mentioned in the pydicom documentation but it seems to work well as seen above
            #png supports 16 gray scale and does not require the PILOW workaround for jpg images
            new_img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            image_bw_arr = np.asarray(new_img)
            clahe = cv2.createCLAHE(clipLimit = 5) 
            clh_img = clahe.apply(image_bw_arr) + 10
            #equ = exposure.equalize_hist(clh_img)
            #final_img=255-equ
            final_img = np.reshape(clh_img, (IMG_SIZE, IMG_SIZE,1))
            final_img = array_to_img(final_img)
            if not os.path.exists(root_directory+base_dir3_clahe+'/'+file_name+'_clahe.jpg'):
                final_img.save(root_directory+base_dir3_clahe+'/'+file_name+'_clahe.jpg')
    if new.loc[int(dir_name)][0]==1 and new.loc[int(dir_name)][1]==1:
        for file_name in files:
            #print('file_name yes effusion yes fatpad')
            ds = pydicom.dcmread(dirpath+'/'+file_name)
            img = ds.pixel_array
            img2d = img.astype(float)
            img2dscaled = (np.maximum(img2d,0)/img2d.max())*255.0
            img2dscaled = np.uint8(img2dscaled)

            #using ds,pixel_array because the pydicom documentation says it will automatically take care of compressed jpg or png,
            #pixel data and also encapsulated pixel data multi-frame volumes in the dicom image
            #cv2 was not mentioned in the pydicom documentation but it seems to work well as seen above
            #png supports 16 gray scale and does not require the PILOW workaround for jpg images
            new_img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            image_bw_arr = np.asarray(new_img)
            clahe = cv2.createCLAHE(clipLimit = 5) 
            clh_img = clahe.apply(image_bw_arr) + 10
            #equ = exposure.equalize_hist(clh_img)
           # final_img=255-equ
            final_img = np.reshape(clh_img, (IMG_SIZE, IMG_SIZE,1))
            final_img = array_to_img(final_img)
            if not os.path.exists(root_directory+base_dir4_clahe+'/'+file_name+'_clahe.jpg'):
                final_img.save(root_directory+base_dir4_clahe+'/'+file_name+'_clahe.jpg')

./data_DICOM\6531039
./data_DICOM\6535531
./data_DICOM\6535558
./data_DICOM\6535615
./data_DICOM\6535855
./data_DICOM\6538876
./data_DICOM\6539704
./data_DICOM\6539909
./data_DICOM\6540440
./data_DICOM\6541856
./data_DICOM\6541916
./data_DICOM\6546122
./data_DICOM\6547136
./data_DICOM\6548728
./data_DICOM\6550251
./data_DICOM\6551761
./data_DICOM\6551975
./data_DICOM\6552239
./data_DICOM\6552939
./data_DICOM\6553963
./data_DICOM\6553981
./data_DICOM\6558229
./data_DICOM\6558527
./data_DICOM\6559861
./data_DICOM\6561796
./data_DICOM\6566611
./data_DICOM\6566728
./data_DICOM\6568013
./data_DICOM\6568022
./data_DICOM\6568192
./data_DICOM\6568282
./data_DICOM\6569872
./data_DICOM\6570171
./data_DICOM\6571781
./data_DICOM\6574524
./data_DICOM\6576136
./data_DICOM\6576245
./data_DICOM\6576257
./data_DICOM\6576313
./data_DICOM\6576319
./data_DICOM\6576529
./data_DICOM\6580657
./data_DICOM\6581301
./data_DICOM\6582753
./data_DICOM\6583297
./data_DICOM\6584450
./data_DICOM\6584640
./data_DICOM\

./data_DICOM\6989936
./data_DICOM\6991438
./data_DICOM\6991459
./data_DICOM\6991461
./data_DICOM\6993056
./data_DICOM\6994397
./data_DICOM\6995017
./data_DICOM\6999370
./data_DICOM\6999702
./data_DICOM\6999748
./data_DICOM\7001445
./data_DICOM\7002849
./data_DICOM\7005148
./data_DICOM\7009169
./data_DICOM\7009948
./data_DICOM\7011458
./data_DICOM\7011926
./data_DICOM\7013100
./data_DICOM\7015589
./data_DICOM\7019864
./data_DICOM\7019894
./data_DICOM\7020256
./data_DICOM\7020284
./data_DICOM\7020512
./data_DICOM\7023756
./data_DICOM\7023824
./data_DICOM\7023935
./data_DICOM\7024755
./data_DICOM\7025604
./data_DICOM\7026854
./data_DICOM\7026855
./data_DICOM\7026865
./data_DICOM\7026951
./data_DICOM\7027207
./data_DICOM\7027581
./data_DICOM\7027668
./data_DICOM\7028330
./data_DICOM\7028332
./data_DICOM\7028339
./data_DICOM\7028515
./data_DICOM\7029095
./data_DICOM\7034809
./data_DICOM\7034813
./data_DICOM\7034846
./data_DICOM\7035036
./data_DICOM\7035168
./data_DICOM\7035184
./data_DICOM\

./data_DICOM\7380950
./data_DICOM\7382556
./data_DICOM\7384127
./data_DICOM\7385148
./data_DICOM\7387326
./data_DICOM\7387369
./data_DICOM\7388945
./data_DICOM\7389221
./data_DICOM\7389285
./data_DICOM\7395483
./data_DICOM\7396466
./data_DICOM\7396906
./data_DICOM\7396912
./data_DICOM\7400038
./data_DICOM\7402469
./data_DICOM\7405481
./data_DICOM\7406935
./data_DICOM\7408637
./data_DICOM\7410071
./data_DICOM\7412026
./data_DICOM\7412046
./data_DICOM\7414879
./data_DICOM\7419849
./data_DICOM\7422135
./data_DICOM\7425842
./data_DICOM\7429900
./data_DICOM\7432685
./data_DICOM\7438415
./data_DICOM\7440133
./data_DICOM\7440196
./data_DICOM\7441678
./data_DICOM\7441739
./data_DICOM\7441751
./data_DICOM\7444661
./data_DICOM\7445073
./data_DICOM\7445188
./data_DICOM\7445435
./data_DICOM\7445449
./data_DICOM\7449044
./data_DICOM\7450861
./data_DICOM\7451720
./data_DICOM\7451762
./data_DICOM\7453791
./data_DICOM\7453876
./data_DICOM\7454612
./data_DICOM\7455650
./data_DICOM\7458431
./data_DICOM\

./data_DICOM\7913949
./data_DICOM\7914527
./data_DICOM\7914658
./data_DICOM\7914793
./data_DICOM\7914796
./data_DICOM\7914819
./data_DICOM\7914942
./data_DICOM\7916123
./data_DICOM\7918483
./data_DICOM\7919582
./data_DICOM\7919645
./data_DICOM\7920632
./data_DICOM\7920723
./data_DICOM\7920748
./data_DICOM\7920771
./data_DICOM\7920898
./data_DICOM\7921771
./data_DICOM\7922386
./data_DICOM\7924075
./data_DICOM\7924126
./data_DICOM\7925259
./data_DICOM\7925314
./data_DICOM\7926497
./data_DICOM\7926662
./data_DICOM\7927689
./data_DICOM\7927795
./data_DICOM\7927853
./data_DICOM\7927956
./data_DICOM\7928261
./data_DICOM\7929918
./data_DICOM\7931242
./data_DICOM\7932744
./data_DICOM\7935829
./data_DICOM\7937615
./data_DICOM\7938744
./data_DICOM\7939468
./data_DICOM\7939789
./data_DICOM\7942520
./data_DICOM\7942977
./data_DICOM\7943103
./data_DICOM\7943147
./data_DICOM\7943189
./data_DICOM\7943227
./data_DICOM\7944716
./data_DICOM\7945887
./data_DICOM\7947494
./data_DICOM\7948871
./data_DICOM\

ValueError: invalid literal for int() with base 10: './data_DICOM'

In [ ]:

base_dir1_clh_bin = '/reformatted_binary_clahe_histeq/No_Effusion'
base_dir2_clh_bin = '/reformatted_binary_clahe_histeq/Yes_Effusion'
base_dir3_clh_bin = '/reformatted_binary_clahe_histeq/Yes_Fracture'
base_dir4_clh_bin = '/reformatted_binary_clahe_histeq/No_Fracture'

if not os.path.exists(root_directory+base_dir1_clh_bin):
    os.makedirs(root_directory+base_dir1_clh_bin)
if not os.path.exists(root_directory+base_dir2_clh_bin):
    os.makedirs(root_directory+base_dir2_clh_bin)
if not os.path.exists(root_directory+base_dir3_clh_bin):
    os.makedirs(root_directory+base_dir3_clh_bin)
if not os.path.exists(root_directory+base_dir4_clh_bin):
    os.makedirs(root_directory+base_dir4_clh_bin)

In [ ]:
#unpack the directory of images which is named radio_images_dataset into a similar named directory
#the walk stops the moment it encounters anything that is not a dicom image
IMG_SIZE=600

for dirpath, dirnames, files in os.walk('./data_DICOM', topdown=False):
    print(dirpath)
    dir_name = dirpath.split(os.path.sep)[-1]
    if new.loc[int(dir_name)][0]==0:
        for file_name in files:
            #print('file_name no effusion no fatpad')
            ds = pydicom.dcmread(dirpath+'/'+file_name)
            img = ds.pixel_array
            img2d = img.astype(float)
            img2dscaled = (np.maximum(img2d,0)/img2d.max())*255.0
            img2dscaled = np.uint8(img2dscaled)

            #using ds,pixel_array because the pydicom documentation says it will automatically take care of compressed jpg or png,
            #pixel data and also encapsulated pixel data multi-frame volumes in the dicom image
            #cv2 was not mentioned in the pydicom documentation but it seems to work well as seen above
            #png supports 16 gray scale and does not require the PILOW workaround for jpg images
            new_img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            image_bw_arr = np.asarray(new_img)
            clahe = cv2.createCLAHE(clipLimit = 15) 
            clh_img = clahe.apply(image_bw_arr) + 35
            histeq_img = exposure.equalize_hist(clh_img)
            final_img = np.reshape(histeq_img, (IMG_SIZE, IMG_SIZE,1))
            final_img = array_to_img(final_img)
            if not os.path.exists(root_directory+base_dir1_clh_bin+'/'+file_name+'_clahe.jpg'):
                final_img.save(root_directory+base_dir1_clh_bin+'/'+file_name+'_clahe.jpg')
    if new.loc[int(dir_name)][0]==1:
        for file_name in files:
            #print('file_name yes effusion no fatpad')
            ds = pydicom.dcmread(dirpath+'/'+file_name)
            img = ds.pixel_array
            img2d = img.astype(float)
            img2dscaled = (np.maximum(img2d,0)/img2d.max())*255.0
            img2dscaled = np.uint8(img2dscaled)

            #using ds,pixel_array because the pydicom documentation says it will automatically take care of compressed jpg or png,
            #pixel data and also encapsulated pixel data multi-frame volumes in the dicom image
            #cv2 was not mentioned in the pydicom documentation but it seems to work well as seen above
            #png supports 16 gray scale and does not require the PILOW workaround for jpg images
            new_img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            image_bw_arr = np.asarray(new_img)
            clahe = cv2.createCLAHE(clipLimit = 15) 
            clh_img = clahe.apply(image_bw_arr) + 35
            histeq_img = exposure.equalize_hist(clh_img)
            final_img = np.reshape(histeq_img, (IMG_SIZE, IMG_SIZE,1))
            final_img = array_to_img(final_img)
            if not os.path.exists(root_directory+base_dir2_clh_bin+'/'+file_name+'_clahe.jpg'):
                final_img.save(root_directory+base_dir2_clh_bin+'/'+file_name+'_clahe.jpg')
    if new.loc[int(dir_name)][1]==1:
        for file_name in files:
            #print('file_name no effusion yes fatpad')
            ds = pydicom.dcmread(dirpath+'/'+file_name)
            img = ds.pixel_array
            img2d = img.astype(float)
            img2dscaled = (np.maximum(img2d,0)/img2d.max())*255.0
            img2dscaled = np.uint8(img2dscaled)


            #using ds,pixel_array because the pydicom documentation says it will automatically take care of compressed jpg or png,
            #pixel data and also encapsulated pixel data multi-frame volumes in the dicom image
            #cv2 was not mentioned in the pydicom documentation but it seems to work well as seen above
            #png supports 16 gray scale and does not require the PILOW workaround for jpg images
            new_img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            image_bw_arr = np.asarray(new_img)
            clahe = cv2.createCLAHE(clipLimit = 15) 
            clh_img = clahe.apply(image_bw_arr) + 35
            histeq_img = exposure.equalize_hist(clh_img)
            final_img = np.reshape(histeq_img, (IMG_SIZE, IMG_SIZE,1))
            final_img = array_to_img(final_img)
            if not os.path.exists(root_directory+base_dir3_clh_bin+'/'+file_name+'_clahe.jpg'):
                final_img.save(root_directory+base_dir3_clh_bin+'/'+file_name+'_clahe.jpg')
    if new.loc[int(dir_name)][1]==0:
        for file_name in files:
            #print('file_name yes effusion yes fatpad')
            ds = pydicom.dcmread(dirpath+'/'+file_name)
            img = ds.pixel_array
            img2d = img.astype(float)
            img2dscaled = (np.maximum(img2d,0)/img2d.max())*255.0
            img2dscaled = np.uint8(img2dscaled)
            

            #using ds,pixel_array because the pydicom documentation says it will automatically take care of compressed jpg or png,
            #pixel data and also encapsulated pixel data multi-frame volumes in the dicom image
            #cv2 was not mentioned in the pydicom documentation but it seems to work well as seen above
            #png supports 16 gray scale and does not require the PILOW workaround for jpg images
            new_img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            image_bw_arr = np.asarray(new_img)
            clahe = cv2.createCLAHE(clipLimit = 15) 
            clh_img = clahe.apply(image_bw_arr) + 35
            histeq_img = exposure.equalize_hist(clh_img)
            final_img = np.reshape(histeq_img, (IMG_SIZE, IMG_SIZE,1))
            final_img = array_to_img(final_img)
            if not os.path.exists(root_directory+base_dir4_clh_bin+'/'+file_name+'_clahe.jpg'):
                final_img.save(root_directory+base_dir4_clh_bin+'/'+file_name+'_clahe.jpg')

In [8]:
sum(len(files) for _,_, files in os.walk(root_directory+base_dir1_clahe))


2539

In [9]:
sum(len(files) for _,_, files in os.walk(root_directory+base_dir4_clahe))

1882

In [10]:
count = sum(len(files) for _,_, files in os.walk('./data_DICOM', topdown=False))
ID_counter = 0
img_id=[]
patient_id=[]
 
for dirpath, dirnames, files in os.walk('./data_DICOM', topdown=False):
    print(dirpath)
    ID_counter+= 1
    dir_name = dirpath.split(os.path.sep)[-1]
    if new.loc[int(dir_name)][0]==0:
        for file_name in files:
            #print('file_name no effusion no fatpad')
            file_id=file_name.split('-')
            img_id.append(file_id[1])
            patient_id.append(ID_counter)
            
           
    if new.loc[int(dir_name)][0]==1:
        for file_name in files:
            #print('file_name yes effusion no fatpad')
            file_id=file_name.split('-')
            img_id.append(file_id[1])
            patient_id.append(ID_counter)
            


./data_DICOM\6531039
./data_DICOM\6535531
./data_DICOM\6535558
./data_DICOM\6535615
./data_DICOM\6535855
./data_DICOM\6538876
./data_DICOM\6539704
./data_DICOM\6539909
./data_DICOM\6540440
./data_DICOM\6541856
./data_DICOM\6541916
./data_DICOM\6546122
./data_DICOM\6547136
./data_DICOM\6548728
./data_DICOM\6550251
./data_DICOM\6551761
./data_DICOM\6551975
./data_DICOM\6552239
./data_DICOM\6552939
./data_DICOM\6553963
./data_DICOM\6553981
./data_DICOM\6558229
./data_DICOM\6558527
./data_DICOM\6559861
./data_DICOM\6561796
./data_DICOM\6566611
./data_DICOM\6566728
./data_DICOM\6568013
./data_DICOM\6568022
./data_DICOM\6568192
./data_DICOM\6568282
./data_DICOM\6569872
./data_DICOM\6570171
./data_DICOM\6571781
./data_DICOM\6574524
./data_DICOM\6576136
./data_DICOM\6576245
./data_DICOM\6576257
./data_DICOM\6576313
./data_DICOM\6576319
./data_DICOM\6576529
./data_DICOM\6580657
./data_DICOM\6581301
./data_DICOM\6582753
./data_DICOM\6583297
./data_DICOM\6584450
./data_DICOM\6584640
./data_DICOM\

./data_DICOM\7046691
./data_DICOM\7049753
./data_DICOM\7051170
./data_DICOM\7051196
./data_DICOM\7051258
./data_DICOM\7051286
./data_DICOM\7051693
./data_DICOM\7054962
./data_DICOM\7054963
./data_DICOM\7054992
./data_DICOM\7058030
./data_DICOM\7058053
./data_DICOM\7058080
./data_DICOM\7059394
./data_DICOM\7059564
./data_DICOM\7059645
./data_DICOM\7059712
./data_DICOM\7059774
./data_DICOM\7059826
./data_DICOM\7060048
./data_DICOM\7064194
./data_DICOM\7065008
./data_DICOM\7065047
./data_DICOM\7065865
./data_DICOM\7066435
./data_DICOM\7066704
./data_DICOM\7066945
./data_DICOM\7069712
./data_DICOM\7070249
./data_DICOM\7070274
./data_DICOM\7070304
./data_DICOM\7071867
./data_DICOM\7072145
./data_DICOM\7072244
./data_DICOM\7072349
./data_DICOM\7073937
./data_DICOM\7074032
./data_DICOM\7074101
./data_DICOM\7074250
./data_DICOM\7075853
./data_DICOM\7075941
./data_DICOM\7077517
./data_DICOM\7078896
./data_DICOM\7080406
./data_DICOM\7080449
./data_DICOM\7081748
./data_DICOM\7081752
./data_DICOM\

./data_DICOM\7525906
./data_DICOM\7526310
./data_DICOM\7527431
./data_DICOM\7528373
./data_DICOM\7530271
./data_DICOM\7533651
./data_DICOM\7533981
./data_DICOM\7534267
./data_DICOM\7534777
./data_DICOM\7534847
./data_DICOM\7535059
./data_DICOM\7535117
./data_DICOM\7535205
./data_DICOM\7535214
./data_DICOM\7536151
./data_DICOM\7536350
./data_DICOM\7536352
./data_DICOM\7536414
./data_DICOM\7537546
./data_DICOM\7538441
./data_DICOM\7538668
./data_DICOM\7540790
./data_DICOM\7541368
./data_DICOM\7541425
./data_DICOM\7541463
./data_DICOM\7542477
./data_DICOM\7542526
./data_DICOM\7542785
./data_DICOM\7544485
./data_DICOM\7547123
./data_DICOM\7548342
./data_DICOM\7549520
./data_DICOM\7550606
./data_DICOM\7550622
./data_DICOM\7552805
./data_DICOM\7553322
./data_DICOM\7553327
./data_DICOM\7553519
./data_DICOM\7554087
./data_DICOM\7555795
./data_DICOM\7558044
./data_DICOM\7558389
./data_DICOM\7558437
./data_DICOM\7559429
./data_DICOM\7560170
./data_DICOM\7564375
./data_DICOM\7567164
./data_DICOM\

ValueError: invalid literal for int() with base 10: './data_DICOM'

In [11]:
df_labelinfo = pd.DataFrame()
df_labelinfo['image_id'] = img_id
df_labelinfo['patient_id'] = patient_id

In [12]:
df_labelinfo['patient_id']

0          1
1          1
2          1
3          1
4          2
        ... 
5023    1306
5024    1307
5025    1307
5026    1307
5027    1307
Name: patient_id, Length: 5028, dtype: int64

In [13]:
#Split each directory into train, validation, and test
base_dir_split = './Binary_Classification_ClaheSplit'

base_dir_train_cl0 = '/Binary_Classification_ClaheSplit/train/EF_cl0'
base_dir_train_cl1 = '/Binary_Classification_ClaheSplit/train/EF_cl1'

base_dir_validate_cl0 = '/Binary_Classification_ClaheSplit/validation/EF_cl0'
base_dir_validate_cl1 = '/Binary_Classification_ClaheSplit/validation/EF_cl1'

base_dir_test_cl0 = '/Binary_Classification_ClaheSplit/test/EF_cl0'
base_dir_test_cl1 = '/Binary_Classification_ClaheSplit/test/EF_cl1'


if not os.path.exists(root_directory+base_dir_split):
    os.mkdir(root_directory+base_dir_split)
    
if not os.path.exists(root_directory+base_dir_train_cl0):
    os.makedirs(root_directory+base_dir_train_cl0)
if not os.path.exists(root_directory+base_dir_train_cl1):
    os.makedirs(root_directory+base_dir_train_cl1)
    
if not os.path.exists(root_directory+base_dir_validate_cl0):
    os.makedirs(root_directory+base_dir_validate_cl0)
if not os.path.exists(root_directory+base_dir_validate_cl1):
    os.makedirs(root_directory+base_dir_validate_cl1)
    
if not os.path.exists(root_directory+base_dir_test_cl0):
    os.makedirs(root_directory+base_dir_test_cl0)
if not os.path.exists(root_directory+base_dir_test_cl1):
    os.makedirs(root_directory+base_dir_test_cl1)

In [14]:
print(base_dir1_clahe)
print(root_directory)

/reformatted_data_clahe/No_Effusion_No_Fracture
C:/Users/Mojtaba Jafari Tadi/Projects_AMK/Radiology Project


In [15]:
###NEGATIVE EFFUSION AND FRACTURE

count_negative = sum(len(files) for _,_, files in os.walk(root_directory+base_dir1_clahe))
counter = 0
train_counter = 0
val_counter = 0
test_counter = 0


for dirpath, dirnames, files in os.walk(root_directory+base_dir1_clahe):
    print(dirpath)
    dir_name = dirpath.split(os.path.sep)[-1]
    for file_name in files:
        counter+=1
        if counter <= (int(0.7*count_negative)):
                shutil.copy2(root_directory+base_dir1_clahe+'/'+file_name, root_directory+base_dir_train_cl0)
                train_counter+=1
                #print("train")
                #print(train_counter)
        if counter > (int(0.7*count_negative)) and counter <= (int(0.9*count_negative)):
                shutil.copy2(root_directory+base_dir1_clahe+'/'+file_name, root_directory+base_dir_validate_cl0)
                val_counter+=1
                #print('val')
                #print(val_counter)
        if counter > (int(0.9*count_negative)):
                shutil.copy2(root_directory+base_dir1_clahe+'/'+file_name, root_directory+base_dir_test_cl0)
                test_counter+=1
                #print('test')
                #print(test_counter)

print(count_negative)   
print(train_counter)
print(val_counter)
print(test_counter)
assert (train_counter+val_counter+test_counter)== count_negative


C:/Users/Mojtaba Jafari Tadi/Projects_AMK/Radiology Project/reformatted_data_clahe/No_Effusion_No_Fracture
2539
1777
508
254


In [16]:
##POSITIVE EFFUSION AND FRACTURE
count_positive = sum(len(files) for _,_, files in os.walk(root_directory+base_dir4_clahe))
counter = 0
train_counter = 0
val_counter = 0
test_counter = 0


for dirpath, dirnames, files in os.walk(root_directory+base_dir4_clahe):
    print(dirpath)
    dir_name = dirpath.split(os.path.sep)[-1]
    for file_name in files:
        counter+=1
        if counter <= (int(0.7*count_positive)):
                shutil.copy2(root_directory+base_dir4_clahe+'/'+file_name, root_directory+base_dir_train_cl1)
                train_counter+=1
                #print("train")
                #print(train_counter)
        if counter > (int(0.7*count_positive)) and counter <= (int(0.9*count_positive)):
                shutil.copy2(root_directory+base_dir4_clahe+'/'+file_name, root_directory+base_dir_validate_cl1)
                val_counter+=1
                #print('val')
                #print(val_counter)
        if counter > (int(0.9*count_positive)):
                shutil.copy2(root_directory+base_dir4_clahe+'/'+file_name, root_directory+base_dir_test_cl1)
                test_counter+=1
                #print('test')
                #print(test_counter)
                
                
    print(train_counter)
    print(val_counter)
    print(test_counter)

C:/Users/Mojtaba Jafari Tadi/Projects_AMK/Radiology Project/reformatted_data_clahe/Yes_Effusion_Yes_Fracture
1317
376
189


In [17]:
#Split each directory into train, validation, and test (EFFUSION + FRACTURE)
base_dir_split = './Binary_Classification_clhhisteq_EFvsnEFSplit'

base_dir_train_cl0 = '/Binary_Classification_clhhisteq_EFvsnEFSplit/train/EFFR_cl0'
base_dir_train_cl1 = '/Binary_Classification_clhhisteq_EFvsnEFSplit/train/EFFR_cl1'

base_dir_validate_cl0 = '/Binary_Classification_clhhisteq_EFvsnEFSplit/validation/EFFR_cl0'
base_dir_validate_cl1 = '/Binary_Classification_clhhisteq_EFvsnEFSplit/validation/EFFR_cl1'

base_dir_test_cl0 = '/Binary_Classification_clhhisteq_EFvsnEFSplit/test/EFFR_cl0'
base_dir_test_cl1 = '/Binary_Classification_clhhisteq_EFvsnEFSplit/test/EFFR_cl1'


if not os.path.exists(root_directory+base_dir_split):
    os.mkdir(root_directory+base_dir_split)
    
if not os.path.exists(root_directory+base_dir_train_cl0):
    os.makedirs(root_directory+base_dir_train_cl0)
if not os.path.exists(root_directory+base_dir_train_cl1):
    os.makedirs(root_directory+base_dir_train_cl1)
    
if not os.path.exists(root_directory+base_dir_validate_cl0):
    os.makedirs(root_directory+base_dir_validate_cl0)
if not os.path.exists(root_directory+base_dir_validate_cl1):
    os.makedirs(root_directory+base_dir_validate_cl1)
    
if not os.path.exists(root_directory+base_dir_test_cl0):
    os.makedirs(root_directory+base_dir_test_cl0)
if not os.path.exists(root_directory+base_dir_test_cl1):
    os.makedirs(root_directory+base_dir_test_cl1)

In [ ]:
##NEGATIVE EFFUSION

count_negative = sum(len(files) for _,_, files in os.walk(root_directory+base_dir1_clh_bin))
counter = 0
train_counter = 0
val_counter = 0
test_counter = 0


for dirpath, dirnames, files in os.walk(root_directory+base_dir1_clh_bin):
    print(dirpath)
    dir_name = dirpath.split(os.path.sep)[-1]
    for file_name in files:
        counter+=1
        if counter <= (int(0.7*count_negative)):
                shutil.copy2(root_directory+base_dir1_clh_bin+'/'+file_name, root_directory+base_dir_train_cl0)
                train_counter+=1
                #print("train")
                #print(train_counter)
        if counter > (int(0.7*count_negative)) and counter <= (int(0.9*count_negative)):
                shutil.copy2(root_directory+base_dir1_clh_bin+'/'+file_name, root_directory+base_dir_validate_cl0)
                val_counter+=1
                #print('val')
                #print(val_counter)
        if counter > (int(0.9*count_negative)):
                shutil.copy2(root_directory+base_dir1_clh_bin+'/'+file_name, root_directory+base_dir_test_cl0)
                test_counter+=1
                #print('test')
                #print(test_counter)

print(count_negative)   
print(train_counter)
print(val_counter)
print(test_counter)
assert (train_counter+val_counter+test_counter)== count_negative


In [ ]:
##POSITIVE EFFUSION

count_positive = sum(len(files) for _,_, files in os.walk(root_directory+base_dir4_clh_bin))
counter = 0
train_counter = 0
val_counter = 0
test_counter = 0


for dirpath, dirnames, files in os.walk(root_directory+base_dir4_clh_bin):
    print(dirpath)
    dir_name = dirpath.split(os.path.sep)[-1]
    for file_name in files:
        counter+=1
        if counter <= (int(0.7*count_positive)):
                shutil.copy2(root_directory+base_dir2_clh_bin+'/'+file_name, root_directory+base_dir_train_cl1)
                train_counter+=1
                #print("train")
                #print(train_counter)
        if counter > (int(0.7*count_positive)) and counter <= (int(0.9*count_positive)):
                shutil.copy2(root_directory+base_dir2_clh_bin+'/'+file_name, root_directory+base_dir_validate_cl1)
                val_counter+=1
                #print('val')
                #print(val_counter)
        if counter > (int(0.9*count_positive)):
                shutil.copy2(root_directory+base_dir2_clh_bin+'/'+file_name, root_directory+base_dir_test_cl1)
                test_counter+=1
                #print('test')
                #print(test_counter)
                
                
    print(train_counter)
    print(val_counter)
    print(test_counter)